## Environment

Run the appropriate "environment" code before running any solutions.

In [4]:
#@title environment: google.colab
from google.colab import files
fread = lambda path: open(path,'r').read()

In [1]:
#@title environment: vscode
fread = lambda path: open('inputs/'+path, 'r').read()

## Imports

Import general-purpose libraries before running any solutions

In [2]:
import re, math
from operator import add
from functools import reduce
from math import prod

class formats:
    PURPLE = '\033[95m'
    BLUE = '\033[94m'
    CYAN = '\033[96m'
    GREEN = '\033[92m'
    ORANGE = '\033[93m'
    RED = '\033[91m'
    WHITE = '\033[0m'
    BOLD = '\033[1m'

# Solutions

Solution inputs should be uploaded with the form `23-{dayNumber}.txt` (e.g., the input for Day 5 would be uploaded as `23-5.txt`). Each solution has the input file name hard-coded into the script, so if you want to run the script against a test file you will need to edit the file name.

## Week 1

In [ ]:
#@title Day 1: Trebuchet?!
ls = fread('23-1.txt').splitlines()

print(sum(map(lambda d: int(d[0]+d[-1]),[[c for c in l if '/' < c < ':'] for l in ls])))

ns = {n: n[0]+str(i)+n[-1] for (i,n) in enumerate('zero,one,two,three,four,five,six,seven,eight,nine'.split(','))}
[ls:= [l.replace(k,ns[k]) for l in ls] for k in ns]
print(sum([int(re.search('\\d',l).group()+re.search('\\d',l[::-1]).group()) for l in ls]))


54573
54591


In [ ]:
#@title Day 2: Cube Conundrum
maxes = {'red': 12, 'green': 13, 'blue': 14}

ls = fread('23-2.txt').splitlines()
gs = [[{p.strip().split(' ')[1]: int(p.strip().split(' ')[0])
        for p in r.split(',')}
        for r in l.split(':')[1].split(';')]
        for l in ls]

valid = [i+1 for (i,g) in enumerate(gs)
             if all([all([maxes[k]>=r[k] for k in r]) for r in g])]
print(sum(valid))

powers = [math.prod([max([r.get(k,0) for r in g]) for k in maxes]) for g in gs]
print(sum(powers))

2727
56580


In [ ]:
#@title Day 3: Gear Ratios
f = fread('23-3.txt').splitlines()

special = lambda x,y,n: [re.search('[^\\d.]',f[min(max(yi,0),len(f)-1)][max(x-1,0):min(x+n+1,len(f[0])-1)]) != None for yi in range(y-1,y+2)]
ps = {(m.start(),y): m.group() for (y,l) in enumerate(f) for m in re.finditer('\\d+',l)}
ns = [int(ps[(x,y)]) for (x,y) in ps if any(special(x,y,len(ps[(x,y)])))]
print(sum(ns))

stars = lambda x,y,n: [
  (m.start(),yi)
  for yi in range(max(0,y-1),min(y+2,len(f)))
  for m in re.finditer('\\*', f[yi]) if (x-2)<m.start()<(x+n+1)
]

ss = [(s, int(ps[(x,y)])) for (x,y) in ps for s in stars(x,y,len(ps[(x,y)]))]
pgs = [[n for t,n in ss if t==u] for u in set([s for s,n in ss])]
gs = [pg for pg in pgs if len(pg)==2]
print(sum([math.prod(g) for g in gs]))


529618
77509019


In [ ]:
#@title Day 4: Scratchcards
f = fread('23-4.txt').splitlines()

cs = [[set([int(n) for n in m.group().split(' ') if n != ''])
  for m in re.finditer('[\\d ]+', l.split(':')[1])]
  for l in f]

wins = [sum([1 for n in c[1] if n in c[0]]) for c in cs]
score = sum([1<<(w-1) for w in wins if w > 0])
print(score)

cards = [1]*len(wins)
[cards:= [sum(n) for n in zip(cards,
  ([0]*(i+1))+([cards[i]]*w)+([0]*(len(wins)-w-i-1)))]
  for i,w in enumerate(wins)]
print(sum(cards))

18619
8063216


In [ ]:
#@title Day 5-1: If You Give A Seed A Fertilizer
f = [s.splitlines() for s in fread('23-5.txt').split('\n\n')]

fn = lambda q,p,r: lambda n: (n-p)+q if p <= n < p + r else None
seeds = list(map(int, f[0][0].split(' ')[1:]))
maps = [[fn(*map(int, l.split(' ')))
  for l in s[1:]]+[lambda n: n]
  for s in f[1:]]

applyone = lambda n, fns: next(g(n) for g in fns if g(n) != None)

def apply(n):
  result = n
  [result := applyone(result, m) for m in maps]
  return result

print(min(apply(n) for n in seeds))

836040384


In [3]:
#@title Day 5-2: If You Give A Seed A Fertilizer
f = [s.splitlines() for s in fread('23-5.txt').split('\n\n')]

class Range:
  def __init__(self, start, end):
    self.start = start
    self.end = end
    self.length = end - start

class AlmanacRange(Range):
  def __init__(self, dest, source, length):
    Range.__init__(self, source, source + length)
    self.__op = lambda n: n + (dest - source)

  def apply(self, r):
    return (
        (set() if self.start <= r.start else {Range(r.start, min(r.end, self.start))})
        .union(set() if self.end >= r.end else {Range(max(r.start, self.end), r.end)}),
        set() if self.end < r.start or r.end < self.start else
        {Range(self.__op(max(self.start, r.start)), self.__op(min(self.end, r.end)))}
        )

class AlmanacMap:
  def __init__(self, name, ranges):
    self.__name = name
    self.__ranges = ranges

  def apply(self, rs):
    acc = (set(rs),{})
    [acc := reduce(
          lambda a,b: (a[0].union(b[0]), a[1].union(b[1])),
          [amr.apply(r) for r in acc[0]] + [(set(),acc[1])])
          for amr in self.__ranges]

    return acc[0].union(acc[1])

class Almanac:
  def __init__(self, maps):
    self.__maps = maps

  def parse(lines):
    return Almanac([
      AlmanacMap(s[0], [AlmanacRange(*map(int, l.split(' '))) for l in s[1:]])
      for s in lines])

  def apply(self, rs):
    acc = set(rs)
    [acc := m.apply(acc) for m in self.__maps]
    return acc

ps = list(map(int, f[0][0].split(' ')[1:]))
seeds = [Range(ps[i], ps[i]+ps[i+1])
    for i in range(0,len(ps),2)]

almanac = Almanac.parse(f[1:])
best = min([r.start for r in almanac.apply(seeds)])
print(best)

10834440


In [100]:
#@title Day 6: Wait For It
f = fread('23-6.txt').splitlines()

solve = lambda t0,t1: [math.ceil((0.5*t0) + (s*0.5*math.sqrt((t0*t0)-(4*t1)))) for s in [1,-1]]
solutions = lambda rs: prod([reduce(lambda a,b: a-b, list(map(math.ceil, solve(*r)))) for r in rs])

races = list(zip(*[map(int, re.split('[ ]+', l)[1:]) for l in f]))
print(solutions(races))

race = tuple(int(l.replace(' ','').split(':')[1]) for l in f)
print(solutions([race]))

800280
45128024
